<a href="https://colab.research.google.com/github/Vikbobvia/Powershell_detection/blob/main/_0_Powershell_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/das-lab/mpsd.git
!git clone https://github.com/mingruimingrui/Convolution-neural-networks-made-easy-with-keras.git
import os
folder_path = "/content/mpsd/malicious_pure"

Cloning into 'mpsd'...
remote: Enumerating objects: 10421, done.
remote: Counting objects: 100% (10421/10421), done.
remote: Compressing objects: 100% (9216/9216), done.
remote: Total 10421 (delta 1200), reused 10418 (delta 1200), pack-reused 0
Receiving objects: 100% (10421/10421), 37.78 MiB | 18.64 MiB/s, done.
Resolving deltas: 100% (1200/1200), done.
Updating files: 100% (12721/12721), done.


In [ ]:

def read_psi_files(folder_path):
  """
  Reads all files with the .psi extension in a folder and prints them line by line.

  Args:
      folder_path: The path to the folder containing the .psi files.
  """
  # Check if folder exists
  if not os.path.isdir(folder_path):
    print(f"Error: Folder '{folder_path}' does not exist.")
    return

  # Loop through files in the folder
  number_loop = 5
  index = 0
  for filename in os.listdir(folder_path):
    if filename.endswith(".ps1"):
      file_path = os.path.join(folder_path, filename)
      # Open the file in read mode
      with open(file_path, 'r') as f:
        # Read all lines and print them
        for line in f:
          print(line.strip())
    index += 1
    if (index >= number_loop) : break

# Example usage
read_psi_files(folder_path)


In [ ]:
import base64
import os

def check_for_base64_encoded_powershell(file_path):
    """Checks if a file contains potential Base64-encoded PowerShell commands."""
    with open(file_path, 'r') as file:
        for line in file:
            if line.lower().startswith(('-e', '-E')):
                    print(f"Potential PowerShell command found (starts with -e or -E): {file_path}")
                    return True
            try:
                # Attempt to decode the line as Base64
                decoded_bytes = base64.b64decode(line)
                decoded_text = decoded_bytes.decode('utf-8')

                # Check for common PowerShell keywords in the decoded text
                if "powershell" in decoded_text.lower():
                    # Potential Base64-encoded PowerShell command found
                    return True



            except Exception:
                # Not a valid Base64 string, continue to the next line
                pass

    return False

def count_encoded_files(folder_path):
    """Counts the number of files containing potential Base64-encoded PowerShell commands."""
    encoded_file_count = 0
    for root, _, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            if check_for_base64_encoded_powershell(file_path):
                encoded_file_count += 1
                print(f"Potential Base64-encoded PowerShell found in: {file_path}")

    return encoded_file_count



In [ ]:

encoded_file_count = count_encoded_files(folder_path)
print(f"Total number of potential Base64-encoded PowerShell files: {encoded_file_count}")


Total number of potential Base64-encoded PowerShell files: 0


In [ ]:

def Creat_folder (folder_path):
  folder_path = folder_path
  os.makedirs(folder_path, exist_ok=True)

In [ ]:
folder_path_preproccess = '/content/preproccessed'
Creat_folder(folder_path_preproccess)
folder_path_cache = '/content/preproccessed/cache'
Creat_folder(folder_path_cache)
folder_path_preproccess_malicious = '/content/preproccessed/malicious_pure_preproccessed'
Creat_folder(folder_path_preproccess_malicious)
cache_file_path = '/content/preproccessed/cache/cache.ps1'



In [ ]:
#Edit file functions


def Copy_file ( source_file_path, dest_file_path):
  # Source and destination file paths
  source_file_path = source_file_path  # Change this to your source file path
  destination_file_path = dest_file_path  # Change this to your destination file path

  # Copying the file using the cp command
  !cp {source_file_path} {destination_file_path}

  print(f'File copied from {source_file_path} to {destination_file_path}')


def Delete_file (file_path):
  file_path = file_path
  os.remove(file_path)
  print(f'File at {file_path} has been deleted')


def Process_file (file_path):
    return
#   # File path to edit
#   file_path = file_path  # Change this to the file path you want to edit

#   # Step 1: Read the content of the file
#   with open(file_path, 'r') as file:
#       content = file.readlines()  # Read the content as a list of lines

#   # Step 2: Modify the content as needed
#   # For example, let's add a comment at the beginning of the file
#   modified_content = ["# This is an edited script\n"] + content

#   # Step 3: Write the modified content back to the file
#   with open(file_path, 'w') as file:
#       file.writelines(modified_content)

#   print(f'File at {file_path} has been edited')


def Save_file (file_path, folder_path, new_name):
  # Define the folder path and file path
  file_path = file_path  # Define the full file path
  folder_path = folder_path  # Change this to the folder path where you want to save the file
  new_name = new_name
  new_file_path = os.path.join(folder_path, new_name)

  # Step 2: Create the folder if it does not exist
  if not os.path.exists(folder_path):
      os.makedirs(folder_path)

  # Step 3: Write the content to the file
  Copy_file(file_path, new_file_path)



def Get_proccessed_file_name (file_path):
    file_path = file_path
    file_name = os.path.basename(file_path)
    new_file_name = file_name.replace('.ps1', '_preprocessed.ps1')
    return new_file_name


In [ ]:
#Edit Cache_file

def Cache_file (file_path):
  Copy_file(file_path, cache_file_path)


def Process_cache_file():
  Process_file(cache_file_path)


def Save_cache_file_to_folder_with_original_file_path(folder_path, original_file_path):
  new_name = Get_proccessed_file_name(original_file_path)
  Save_file(cache_file_path, folder_path, new_name)



def Delete_cache_file ():
  try :
   Delete_file(cache_file_path)

  except FileNotFoundError:
    # This block will execute if the file is not found
    print(f'Error: The file at {file_path} was not found.')



In [ ]:
file_path = '/content/mpsd/malicious_pure/1.ps1'
new_folder_path = "/content/preproccessed/malicious_pure_preproccessed"
Cache_file(file_path)
Process_cache_file()
Save_cache_file_to_folder_with_original_file_path(new_folder_path, file_path)
Delete_cache_file()


File copied from /content/mpsd/malicious_pure/1.ps1 to /content/preproccessed/cache/cache.ps1
File copied from /content/preproccessed/cache/cache.ps1 to /content/preproccessed/malicious_pure_preproccessed/1_preprocessed.ps1
File at /content/preproccessed/cache/cache.ps1 has been deleted


In [ ]:
import os

def count_files(directory):
    count = 0
    for root, _, files in os.walk(directory):
        count += len(files)
    return count

# Replace 'path_to_your_folder' with the path to the folder you want to count files in
folder_path = '/content/mpsd/malicious_pure'
file_count = count_files(folder_path)
print("Number of files in the folder:", file_count)


Number of files in the folder: 4202


In [ ]:
def Process_a_file(file_path, new_folder_path):
    Cache_file(file_path)
    Process_cache_file()
    Save_cache_file_to_folder_with_original_file_path(new_folder_path, file_path)
    Delete_cache_file()



def Process_all_of_folder(folder_path, new_folder_path):
   for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = (os.path.join(root, file))
            Process_a_file(file_path, new_folder_path)



def Create_all_Process_folder (unprocessed_folder_path, preproccessed_folder_path):
    folders = []
    for item in os.listdir(unprocessed_folder_path):
        # Check if the item is a directory
        if os.path.isdir(os.path.join(unprocessed_folder_path, item)) and not item.startswith('.'):
          item = item + "_preproccessed"
          folders.append(item)

    for item in folders:
        print(item)

    try:
      for item in folders:
          new_folder_path = os.path.join(preproccessed_folder_path, item)
          # Check if the folder already exists
          if not os.path.exists(new_folder_path):
              # If it doesn't exist, create the folder
              os.makedirs(new_folder_path)
              print("Folder created at", new_folder_path)

          else:
              print("Folder already exists at", new_folder_path)

    except Exception as e:
          print("Error:", e)





def Process_mpsd(unprocessed_folder_path, preproccessed_folder_path):
      Create_all_Process_folder(unprocessed_folder_path, preproccessed_folder_path)
      for item in os.listdir(unprocessed_folder_path):
        X_folder_path = os.path.join(unprocessed_folder_path ,item)
        item = item + "_preproccessed"
        Y_folder_path = os.path.join(preproccessed_folder_path , item)



        if os.path.isdir(X_folder_path ) and os.path.isdir (Y_folder_path) and len(os.listdir(Y_folder_path)) == 0  :
          print("X_folder_path = ", X_folder_path )
          print("Y_folder_path = ", Y_folder_path  )
          Process_all_of_folder(X_folder_path, Y_folder_path)



Process_mpsd("/content/mpsd", "/content/preproccessed")

In [ ]:
Creat_folder("/content/test")

In [ ]:
Process_a_file("/content/mpsd/malicious_pure/1000.ps1", '/content/test')
Process_a_file("/content/mpsd/malicious_pure/1001.ps1", '/content/test')
Process_a_file("/content/mpsd/malicious_pure/1002.ps1", '/content/test')
Process_a_file("/content/mpsd/malicious_pure/1003.ps1", '/content/test')


File copied from /content/mpsd/malicious_pure/1000.ps1 to /content/preproccessed/cache/cache.ps1
File copied from /content/preproccessed/cache/cache.ps1 to /content/test/1000_preprocessed.ps1
File at /content/preproccessed/cache/cache.ps1 has been deleted
File copied from /content/mpsd/malicious_pure/1001.ps1 to /content/preproccessed/cache/cache.ps1
File copied from /content/preproccessed/cache/cache.ps1 to /content/test/1001_preprocessed.ps1
File at /content/preproccessed/cache/cache.ps1 has been deleted
File copied from /content/mpsd/malicious_pure/1002.ps1 to /content/preproccessed/cache/cache.ps1
File copied from /content/preproccessed/cache/cache.ps1 to /content/test/1002_preprocessed.ps1
File at /content/preproccessed/cache/cache.ps1 has been deleted
File copied from /content/mpsd/malicious_pure/1003.ps1 to /content/preproccessed/cache/cache.ps1
File copied from /content/preproccessed/cache/cache.ps1 to /content/test/1003_preprocessed.ps1
File at /content/preproccessed/cache/cac

In [ ]:
import re
import binascii

def get_file_content_and_replace_number(filename):
  """
  This function reads the content of a text file and stores it in a variable.

  Args:
      filename: The path to the text file.

  Returns:
      A string containing the file content, or None if there's an error.
  """


  try:
    with open(filename, 'r') as f:
      content = f.read()


      #number to *
      str_text = re.sub(r'\d+', '*', content)
      str_text = re.sub(r'\s+', ' ', str_text)
      # print(str_text)
    with open(filename, 'w') as f:
      f.write(str_text)
      return str_text
  except FileNotFoundError:
    print(f"Error: File not found - {filename}")
    return None





text = get_file_content_and_replace_number("/content/test/1000_preprocessed.ps1")
text = get_file_content_and_replace_number("/content/test/1001_preprocessed.ps1")
text = get_file_content_and_replace_number("/content/test/1002_preprocessed.ps1")
text = get_file_content_and_replace_number("/content/test/1003_preprocessed.ps1")




